In [559]:
import os
import openai
import sys
import json
import re
from IPython.display import display, HTML
sys.path.append('../..')

In [560]:
openai.api_key = os.environ['OPENAI_API_KEY']

In [561]:
gpt_version = "gpt-3.5-turbo"

## LangChain Imports

In [562]:
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
# function tools
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_functions
# routing/state/chain
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough
# memory
from langchain.memory import ConversationBufferMemory
# agents
from langchain.agents import AgentExecutor

## Component and Templates

In [563]:
card_html = '''
```
<!doctype html>
<html lang="en">
  <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <meta name="description" content="">
    <meta name="author" content="">
  

    <title>Bootstrap Table</title>
    
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-QWTKZyjpPEjISv5WaRU9OFeRpok6YctnYmDr5pNlyT2bRjXh0JMhjY6hW+ALEwIH" crossorigin="anonymous">
   </head>
      <body>
        <div class="row">
            <!-- other classes like col-sm-6 can be used to adjust size-->
            <!--      example of one card -->
          <div class="col-sm-4">
            <div class="card">
              <div class="card-body">
                <h5 class="card-title">Special title treatment</h5>
                <p class="card-text">With supporting text below as a natural lead-in to additional content.</p>
                <!-- replace with link to webpage with related content (similar to a "see more") -->
                <a href="#" class="btn btn-primary">Go somewhere</a>
              </div>
            </div>
          </div>
            <!--      add more cards or delete -->
        </div>
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/js/bootstrap.bundle.min.js" integrity="sha384-YvpcrYf0tY3lHB60NNkmXc5s9fDVZLESaAA55NDzOxhy9GkcIdslK1eN7N6jIeHz" crossorigin="anonymous"></script>
    </body>
</html>
```
'''

In [564]:
table_html = '''
<!doctype html>
<html lang="en">
  <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <meta name="description" content="">
    <meta name="author" content="">
  

    <title>Bootstrap Table</title>
    
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-QWTKZyjpPEjISv5WaRU9OFeRpok6YctnYmDr5pNlyT2bRjXh0JMhjY6hW+ALEwIH" crossorigin="anonymous">
   </head>
  <body>
    <div class="table-responsive">
    <table class="table table-hover">
      <thead class="table-dark">
        <tr>
          <!--  Add More Column Headings or Change Heading Text -->
          <th scope="col">#</th>
          <th scope="col">First</th>
          <th scope="col">Second</th>
          <th scope="col">Third</th>
        </tr>
      </thead>
      <tbody>
        <tr>
          <!--  Change Cell Text -->
          <th scope="row">1</th>
          <td>Text</td>
          <td>Text</td>
          <td>Text</td>
        </tr>
        <tr>
          <th scope="row">2</th>
          <td>Text</td>
          <td>Text</td>
          <td>Text</td>
        </tr>
        <tr>
          <th scope="row">3</th>
          <td>Text</td>
          <td>Text</td>
          <td>Text</td>
        </tr>
      </tbody>
    </table>
    
    <table class="table table-hover">
      <thead class="table-light">
        <tr>
          <th scope="col">#</th>
          <th scope="col">First</th>
          <th scope="col">Second</th>
          <th scope="col">Third</th>
        </tr>
      </thead>
      <tbody>
        <tr>
          <th scope="row">1</th>
          <td>Text</td>
          <td>Text</td>
          <td>Text</td>
        </tr>
        <tr>
          <th scope="row">2</th>
          <td>Text</td>
          <td>Text</td>
          <td>Text</td>
        </tr>
        <tr>
          <th scope="row">3</th>
          <td>Text</td>
          <td>Text</td>
          <td>Text</td>
        </tr>
        <!--   Delete or Add more rows -->
      </tbody>
        <!--   Add more tables -->
      </div>
      <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/js/bootstrap.bundle.min.js" integrity="sha384-YvpcrYf0tY3lHB60NNkmXc5s9fDVZLESaAA55NDzOxhy9GkcIdslK1eN7N6jIeHz" crossorigin="anonymous"></script>
      </body>
    </table>
</html>
'''

In [565]:
table_instructions = '''
Make sure text cells look interactive and clean.
Provide the option on the header cells to sort by column.

Make sure the number of rows and columns align with the content.

Data tables should contain:
    - Interactive components (such as chips, buttons, or menus)
    - Non-interactive elements (such as badges)
    - Tools to query and manipulate data such
'''

In [566]:
def html_instructions(component_specific_instructions = ""):
    return f'''You are given a HTML template in triple backticks. 
    Your task is to use the HTML template code and generate ONLY valid HTML code such that the user input is being used as the content.
    Valid HTML must start with <!DOCTYPE html> tag. You should adjust the CSS classes used to adapt to the content and
    when modifying the CSS/style of the template, you should use bootstrap classes.
    
    {component_specific_instructions}
    
    User Input (content):
    '''

## Tools

In [567]:
html_pattern = re.compile(r'<!DOCTYPE html>.*?</html>', re.DOTALL)

In [568]:
def render_html(code):
    html_match = html_pattern.search(code)
    if html_match:
        display(HTML(html_match.group(0)))

In [575]:
@tool
def make_card(information: str) -> str:
    """A card is helpful when you want to display several to many options or suggestions to the user and 
    provide visuals and descriptions for each option.

    Use a card to display content and actions on a single topic.

    Cards should be easy to scan for relevant and actionable information. 
    Elements like text and images should be placed on cards in a way that clearly indicates hierarchy.

    The output is HTML code.
    """
    print("card tool called")
    prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=card_html + html_instructions()
        ),  # The persistent system prompt
        
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will injected
    ])

    llm = ChatOpenAI(model_name=gpt_version, temperature = 0)
    chat_llm_chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=False,
    )
    o = chat_llm_chain.predict(human_input=information)
    render_html(o)
    return "cards are generated"

In [576]:
@tool
def make_table(information: str) -> str:
    """Tables can be a useful for presenting related content. A table is made of cells organized in rows and columns. 
    Heading cells tell you about the information they introduce. Table data cells provide the specific information.

    Data tables display information in a grid-like format of rows and columns. 
    They organize information in a way that’s easy to scan so that users can look for patterns and develop insights from data.

    The output is HTML code for the Table.
    """
    print("table tool called")
    prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=table_html + html_instructions(table_instructions)
        ),  # The persistent system prompt
        
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will injected
    ])

    llm = ChatOpenAI(model_name=gpt_version, temperature = 0)
    chat_llm_chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=False,
    )
    o = chat_llm_chain.predict(human_input=information)
    render_html(o)
    return "table is generated"

## Setup and Initialize Chain

In [577]:
# this is the directions for the main model with the 
main_system_template = """be kind and helpful to the user. You have tools to assist the user in having a better conversation with you.
"""

In [578]:
def make_agent(system_template, verbose_mode=False):

    mem = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

    tools = [make_card, make_table]
    functions = [ convert_to_openai_function(f) for f in tools]

    model = ChatOpenAI(model_name=gpt_version, temperature=0).bind(functions=functions)
    prompt = ChatPromptTemplate.from_messages([ ("system", system_template),
        MessagesPlaceholder(variable_name="chat_history"),("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")])

    agent_chain = RunnablePassthrough.assign(agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])) | prompt | model | OpenAIFunctionsAgentOutputParser()                                              
    agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=verbose_mode, memory=mem)

    return agent_executor

In [579]:
def interact(agent, input):
    o = agent.invoke({"input": input})
    # print(o)
    o = o['output']
    
    # html_match = html_pattern.search(o)
    # s = False
    # if html_match:
    #     s = True
    #     display(HTML(html_match.group(0)))
    # if s:
    #     o = re.sub(r"```html.*?```", "", o, flags=re.DOTALL)
    return o
    

## Testing

In [580]:
agent = make_agent(main_system_template, verbose_mode=False)

In [581]:
interact(agent, "hi there, my name is Z. ")

'Hello Z! How can I assist you today?'

In [582]:
interact(agent, "I want 4 options for writing style")

card tool called


'Here are 4 options for writing styles:\n\n1. Formal Writing Style\n2. Informal Writing Style\n3. Academic Writing Style\n4. Creative Writing Style\n\nLet me know if you need more information on any of these styles!'

In [583]:
interact(agent, "Think of 5 movies to watch. Give me a description and overview of each. Image would be nice")

card tool called


"Here are 5 movie recommendations with descriptions and overviews:\n\n1. **The Shawshank Redemption**\n   - Description: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.\n   - Overview: A powerful drama that explores friendship, hope, and the human spirit.\n   ![The Shawshank Redemption](https://upload.wikimedia.org/wikipedia/en/8/81/ShawshankRedemptionMoviePoster.jpg)\n\n2. **Inception**\n   - Description: A thief who enters the dreams of others to steal their secrets must plant an idea into a CEO's mind.\n   - Overview: A mind-bending thriller that challenges the concept of reality and dreams.\n   ![Inception](https://upload.wikimedia.org/wikipedia/en/2/2e/Inception_%282010%29_theatrical_poster.jpg)\n\n3. **The Godfather**\n   - Description: The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.\n   - Overview: A classic crime drama that delves into famil

In [584]:
interact(agent, '''Here are grades for some of my students. I want to sort them.

John: Exam Grades [85, 90, 88, 87, 92], Final Grade: 89
Emily: Exam Grades [78, 82, 79, 85, 87], Final Grade: 82
Michael: Exam Grades [90, 88, 92, 87, 85], Final Grade: 88
Sarah: Exam Grades [75, 80, 77, 82, 79], Final Grade: 78
David: Exam Grades [85, 84, 88, 86, 90], Final Grade: 87
Jessica: Exam Grades [92, 95, 90, 88, 93], Final Grade: 92
Ryan: Exam Grades [80, 82, 85, 81, 84], Final Grade: 82
Samantha: Exam Grades [88, 90, 86, 89, 92], Final Grade: 89
Matthew: Exam Grades [85, 87, 83, 88, 90], Final Grade: 87
Amanda: Exam Grades [82, 85, 80, 84, 81], Final Grade: 82
''')

'I can help you sort the students based on their final grades. Here is the sorted list:\n\n1. Jessica: Final Grade - 92\n2. John: Final Grade - 89\n3. Samantha: Final Grade - 89\n4. Michael: Final Grade - 88\n5. David: Final Grade - 87\n6. Matthew: Final Grade - 87\n7. Emily: Final Grade - 82\n8. Ryan: Final Grade - 82\n9. Amanda: Final Grade - 82\n10. Sarah: Final Grade - 78\n\nIf you need any more assistance or information, feel free to ask!'

In [585]:
interact(agent, '''visually sort them by final grade''')

table tool called


'Here is a visual representation of the students sorted by their final grades:\n\n| Student   | Final Grade |\n|-----------|-------------|\n| Jessica   | 92          |\n| John      | 89          |\n| Samantha  | 89          |\n| Michael   | 88          |\n| David     | 87          |\n| Matthew   | 87          |\n| Emily     | 82          |\n| Ryan      | 82          |\n| Amanda    | 82          |\n| Sarah     | 78          |\n\nI hope this visual sorting helps! Let me know if you need any more assistance.'

In [586]:
interact(agent, '''thank you for the help. I want to make business cards but unsure about how to design them''')

"I can provide you with some tips on designing effective business cards. Here are a few key points to consider:\n\n1. **Keep it Simple**: Make sure your business card is clean and uncluttered. Include essential information like your name, job title, contact information, and company logo.\n\n2. **Choose the Right Font**: Use a legible font that reflects your brand's personality. Avoid using too many different fonts on one card.\n\n3. **Use High-Quality Images**: If you include images or a logo, make sure they are high-resolution for a professional look.\n\n4. **Consider Card Size and Shape**: Standard business card size is 3.5 x 2 inches, but you can also opt for unique shapes or sizes to stand out.\n\n5. **Include White Space**: Leave some empty space on the card to make it easier to read and more visually appealing.\n\n6. **Use Colors Wisely**: Choose colors that align with your brand and are visually appealing. Stick to a color scheme that complements your logo.\n\n7. **Double-Check 

In [587]:
interact(agent, '''what are some examples of business cards?''')

card tool called


'Here are some examples of business card designs:\n\n1. **Minimalist Design**\n   - Clean and simple design with essential information.\n\n2. **Creative Shape**\n   - Unique shape or die-cut design to make a statement.\n\n3. **Photographic Background**\n   - Incorporating a high-quality image as the background.\n\n4. **Embossed or Foil Details**\n   - Adding texture and elegance with embossed or foil details.\n\n5. **Vertical Orientation**\n   - Opting for a vertical layout for a modern look.\n\nThese are just a few examples to inspire your business card design. Let me know if you need more ideas or assistance with designing your business cards!'